In [1]:
!nvidia-smi

Fri Jan 27 12:17:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   40C    P0    53W / 300W |      0MiB / 32508MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2023-01-27 12:17:42.901861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 12:17:43.065859: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8289998815867372505
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 32476168192
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7446379011003645677
physical_device_desc: "device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0e:00.0, compute capability: 7.0"
xla_global_id: 416903419
]


2023-01-27 12:17:46.211615: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 12:17:49.513343: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0e:00.0, compute capability: 7.0


In [4]:
import gc
import gdown
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import tqdm.notebook as tqdm

In [5]:
aspect = ['cellular','component']
aspect_abbr = 'cc'

In [6]:
file = 'cafa3_train/cc_df.csv'
# if not os.path.exists(file):
#     url = "https://drive.google.com/file/d/1RyeLQPFTMWAIr-OzELTWIx60ln-mZ7g_/view?usp=sharing"
#     output = file
#     gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [7]:
df = pd.read_csv(file)
df

,Entry,Sequence,Gene Ontology (cellular component)
0,A0A060X6Z0,MPISSSSSSSTKSMRRAASELERSDSVTSPRFIGRRQSLIEDARKE...,GO:0098805;GO:0071944;GO:0005575;GO:0005623;GO...
1,A0A068FIK2,MEVGGGSEECCVKVAVHVRPLIGDEKVQGCKDCVTVIPGKPQVQIG...,GO:0005623;GO:0005856;GO:0043232;GO:0005881;GO...
2,A0A075F932,MVSESHHEALAAPPATTVAAAPPSNVTEPASPGGGGGKEDAFSKLK...,GO:0005575;GO:0005623
3,A0A078CGE6,MARQMTSSQFHKSKTLDNKYMLGDEIGKGAYGRVYIGLDLENGDFV...,GO:0044428;GO:0005623;GO:0043233;GO:0043232;GO...
4,A0A087X1C5,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,GO:0005575;GO:0043231;GO:0005623;GO:0044464;GO...
...,...,...,...
50591,U3JAG9,MHTTRSPSASIQAGAAGDALDLSLNGSQLTMGRRPSSASPGKHFSR...,GO:0098805;GO:0044459;GO:0005623;GO:0005575;GO...
50592,U4PR86,MSKYEVLQGFYAVHDELGSGGFGKVRLATHLLTNQKVAIKIIDKKQ...,GO:0005813;GO:0005575;GO:0005623;GO:0044464;GO...
50593,V5JFY4,MGPWTLLLLHLPLVVSMLPAPTNVSIVSFNLEHTLTWLPGPETPDN...,GO:0016020;GO:0005575
50594,V9GXG1,MPYAEITVNLGKVTLGEENRKKMTNSCLKRHENSSLVQAVCALLNS...,GO:0005575;GO:0043231;GO:0005623;GO:0044464;GO...


In [8]:
go_terms_bp = set()
for idx, row in df.iterrows():
    for term in row['Gene Ontology ('+' '.join(aspect)+')'].split(';'):
        go_terms_bp.add(term)
go_terms_bp = list(go_terms_bp)
go_terms_bp.sort()
print(len(go_terms_bp))
# print(go_terms_bp)

551


In [9]:
def get_segments(sequence,segment_size=100,gap=30):
    segments = []
    start = 0
    end = segment_size
    while end <= len(sequence):
        segments.append(sequence[start:end])
        start += gap
        end += gap
    last_segment = sequence[start:]
    segments.append(last_segment)
    
    return segments

def get_training_data(df,segment_size=100,gap=30):
    training_data = list()
    for idx,row in tqdm.tqdm(df.iterrows()):
        labels = [0] * len(go_terms_bp)
        for term in row['Gene Ontology ('+' '.join(aspect)+')'].split(';'):
            labels[go_terms_bp.index(term)] = 1
        segments = get_segments(row['Sequence'],segment_size,gap)
        for segment in segments:
            training_data.append([row['Entry'],segment,labels])
    return training_data

In [34]:
training_data = get_training_data(df,gap=20)
print(len(training_data))

0it [00:00, ?it/s]

1210233


In [35]:
def get_ngrams(segment,n=3):
    ngrams = []
    for i in range(len(segment)-n+1):
        ngrams.append(segment[i:i+n])
    return ngrams

In [36]:
# Generate training data of ngrams
# if os.path.exists('bp/training_data_4grams.npy'):
#     print('Loading saved ngrams...')
#     training_data_ngrams = np.load('bp/training_data_4grams.npy',allow_pickle=True)
# else:
print('Preparing from scratch...')
training_data_ngrams = []

for i in tqdm.tqdm(range(len(training_data))):
    training_data_ngrams.append([training_data[i][0],get_ngrams(training_data[i][1],n=4),training_data[i][2]])
        
#     np.save('bp/training_data_4grams.npy',training_data_ngrams)
    
print(len(training_data_ngrams))

Preparing from scratch...


  0%|          | 0/1210233 [00:00<?, ?it/s]

1210233


In [37]:
def get_skip_grams(segment,skip=1,n=3):
    skip_grams = []
    window_size = skip + n
    for i in range(len(segment)-window_size+1):
        window = segment[i:i+window_size]
        indices = list(range(window_size))
        indices.pop(0)
        for idx in indices[::-1]:
            temp = ''
            for j in range(window_size):
                if j!=idx:
                    temp+=window[j]
            skip_grams.append(temp)

    return skip_grams

In [38]:
# if os.path.exists('bp/training_data_skip1_4grams.npy'):
#     print('Loading saved skip grams...')
#     training_data_skip_grams = np.load('bp/training_data_skip1_4grams.npy',allow_pickle=True)
# else:
print('Preparing from scratch...')
training_data_skip_grams = []
for i in tqdm.tqdm(range(len(training_data))):
    training_data_skip_grams.append([training_data[i][0],get_skip_grams(training_data[i][1],n=4),training_data[i][2]])
#     np.save('bp/training_data_skip1_4grams.npy',training_data_skip_grams)
print(len(training_data_skip_grams))

Preparing from scratch...


  0%|          | 0/1210233 [00:00<?, ?it/s]

1210233


In [39]:
import tensorflow as tf
import tensorflow_addons as tfa

In [40]:
#Assuming training_data as global variable

def train_test_split(X,y,fold_no,prev_index,Kfolds=5):
    test_split = 1/Kfolds
    
    start_index = prev_index
    end_index = (fold_no + 1) * (test_split) * len(X)
    end_index = round(end_index)
    
    if end_index==len(X):
        end_index -= 1
    
    entry = training_data[end_index][0]
    entries = [sample[0] for sample in training_data]
    
    first_occurence = entries.index(entry)
    entries.reverse()
    
    last_occurence = entries.index(entry)
    last_occurence = len(entries) - last_occurence - 1
    
    del entries
    gc.collect()
    
    end_index = first_occurence if (abs(end_index-first_occurence) < abs(end_index-last_occurence)) else last_occurence
    
    X_test = X[start_index:end_index+1]
    y_test = y[start_index:end_index+1]
    X_train = X[:start_index]
    X_train.extend(X[end_index+1:])
    y_train = y[:start_index]
    y_train.extend(y[end_index+1:])
    
    return X_train, y_train, X_test, y_test, start_index, end_index + 1

In [41]:
MAX_WORDS = 331776
MAX_LEN_NG = 97 #100
MAX_LEN_SG = 384 #300

def tokenization(X_train,X_test,maxlen):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_WORDS)
    tokenizer.fit_on_texts(X_train)

    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

    vocab_size = len(tokenizer.word_index) + 1

    X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=maxlen)
    
    return X_train, X_test, vocab_size, tokenizer

In [42]:
from tensorflow.keras import backend as K

false_negative_penalty = 6
false_positive_penalty = 1

def custom_loss(y_true, y_logit):

    loss = float(0)
    y_true = tf.cast(y_true, tf.float32)
    y_logit = tf.cast(y_logit, tf.float32)
    
    first_term = false_negative_penalty * float(y_true) * - K.log(y_logit + K.epsilon())
    second_term = false_positive_penalty * (1 - float(y_true)) * - K.log(1 - y_logit + K.epsilon())
    
    loss = K.mean(first_term+second_term)

    return loss

def precision(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=1)
    precision = true_positives / (predicted_positives + K.epsilon())
    
    return K.mean(precision)


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=1)
    recall = true_positives / (possible_positives + K.epsilon())
    return K.mean(recall)

def f1_score(y_true, y_pred):
    rec = recall(y_true,y_pred)
    prec = precision(y_true,y_pred)
    f1 = 2*prec*rec/(prec+rec)
    return f1

In [43]:
class attention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences=True,**kwargs):
        self.return_sequences = return_sequences
        super(attention,self).__init__()

    def get_config(self):
      config = super().get_config().copy()
      config.update({
          'return_sequences': self.return_sequences 
      })
      return config
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [44]:
NUM_CLASSES = 551 #For cc (Change according to aspects)

def get_model_ng_sg(vocab_size_ng, vocab_size_sg):
    #Input layers

    input_ngrams = tf.keras.layers.Input(shape=(MAX_LEN_NG,)) 
    input_skip_grams = tf.keras.layers.Input(shape=(MAX_LEN_SG,)) 

    #embeddings
    embedding_layer_ngrams = tf.keras.layers.Embedding(vocab_size_ng, 32)(input_ngrams)
    embedding_layer_skip_grams = tf.keras.layers.Embedding(vocab_size_sg, 32)(input_skip_grams)

    #BI-LSTMs for each of the inputs
    sequence_output_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True))(embedding_layer_ngrams)
    attention_output_1 = attention(return_sequences=False)(sequence_output_1)
    dropout_1 = tf.keras.layers.Dropout(0.3)(attention_output_1)
    dense_layer_1 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_1)

    sequence_output_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70, return_sequences=True))(embedding_layer_skip_grams)
    attention_output_2 = attention(return_sequences=False)(sequence_output_2)
    dropout_2 = tf.keras.layers.Dropout(0.3)(attention_output_2)
    dense_layer_2 = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dropout_2)

    max_layer = tf.keras.layers.Maximum()([dense_layer_1,dense_layer_2])

    model = tf.keras.models.Model(
        inputs=[
            input_ngrams,
            input_skip_grams
        ], 
        outputs=max_layer)
    
    return model

In [45]:
def evaluate_annotations(real_annots, pred_annots):
    total = 0
    p = 0.0
    r = 0.0
    p_total= 0
    fps = []
    fns = []
    for i in range(len(real_annots)):
        if len(real_annots[i]) == 0:
            continue
        tp = set(real_annots[i]).intersection(set(pred_annots[i]))
        fp = set(pred_annots[i]) - tp
        fn = set(real_annots[i]) - tp
        
        fps.append(fp)
        fns.append(fn)
        tpn = len(tp)
        fpn = len(fp)
        fnn = len(fn)
        total += 1
        recall = tpn / (1.0 * (tpn + fnn))
        r += recall
        if len(pred_annots[i]) > 0:
            p_total += 1
            precision = tpn / (1.0 * (tpn + fpn))
            p += precision
    r /= total
    if p_total > 0:
        p /= p_total
    f = 0.0
    if p + r > 0:
        f = 2 * p * r / (p + r)
    return f, p, r, fps, fns

### Model using skip grams and ngrams

In [46]:
#Considering ngrams and skip grams
X_train_ng = [' '.join(sample[1]) for sample in training_data_ngrams]
X_train_sg = [' '.join(sample[1]) for sample in training_data_skip_grams]
y_train = [sample[2] for sample in training_data]

del training_data_ngrams
del training_data_skip_grams

In [47]:
print(len(X_train_ng),len(X_train_sg))

1210233 1210233


In [48]:
test_df = pd.read_csv('cafa3_test/cc_df.csv')
test_df

,Entry,Sequence,Gene Ontology (cellular component)
0,T100900000026,MAESFKELDPDSSMGKALEMTCAIQNQLARILAEFEMTLERDVLQP...,GO:0005911;GO:0005575;GO:0043296;GO:0030054;GO...
1,T100900000115,MNNLSFSELCCLFCCPPCPGKIASKLAFLPPDPTYTLMCDESGSRW...,GO:0005623;GO:0098794;GO:0005886;GO:0043227;GO...
2,T100900000116,MPEPGPRMNGFSLGELCWLFCCPPCPSRIAAKLAFLPPEPTYTVLA...,GO:0005575;GO:0005623;GO:0044464;GO:0044456;GO...
3,T100900000167,MEKSWMLWSFIERWLLALASWSWALCRISLLPLIVTFHLYGGIVLL...,GO:0005623;GO:0043005;GO:0032838;GO:0005886;GO...
4,T100900000453,MRIGLLWLVPLFTLTEGTDGFLQQKNDGRRTKEIVSMVEERHPVHE...,GO:0005575;GO:0005623;GO:0044464;GO:0016020;GO...
...,...,...,...
1260,T992870000225,MVLGKPQTDPTLEWFLSHCHIHKYPSKSTLIHQGEKAETLYYIVKG...,GO:0032993;GO:0032991;GO:0005575
1261,T992870000466,MIPEKRIIRRIQSGGCAIHCQDCSISQLCIPFTLNEHELDQLDNII...,GO:0032993;GO:0032991;GO:0005575
1262,T992870000685,MPEVKTEKPHLLDMGKPQLRMVDLNLLTVFDAVMQEQNITRAAHTL...,GO:0032993;GO:0032991;GO:0005575
1263,T992870001023,MMRVLVVEDNALLRHHLKVQLQDSGHQVDAAEDAREADYYLNEHLP...,GO:0032993;GO:0032991;GO:0005575


In [49]:
testing_data = get_training_data(test_df,gap=20)
print(len(testing_data))

0it [00:00, ?it/s]

30372


In [50]:
print('Preparing from scratch...')
testing_data_ngrams = []

for i in tqdm.tqdm(range(len(testing_data))):
    testing_data_ngrams.append([testing_data[i][0],get_ngrams(testing_data[i][1],n=4),testing_data[i][2]])
    
print(len(testing_data_ngrams))

Preparing from scratch...


  0%|          | 0/30372 [00:00<?, ?it/s]

30372


In [51]:
print('Preparing from scratch...')
testing_data_skip_grams = []

for i in tqdm.tqdm(range(len(testing_data))):
    testing_data_skip_grams.append([testing_data[i][0],get_skip_grams(testing_data[i][1],n=4),testing_data[i][2]])
    
print(len(testing_data_skip_grams))

Preparing from scratch...


  0%|          | 0/30372 [00:00<?, ?it/s]

30372


In [52]:
#Considering ngrams and skip grams
X_test_ng = [' '.join(sample[1]) for sample in testing_data_ngrams]
X_test_sg = [' '.join(sample[1]) for sample in testing_data_skip_grams]
y_test = [sample[2] for sample in testing_data]

del testing_data_ngrams
del testing_data_skip_grams

In [53]:
print(len(X_test_ng),len(X_test_sg))

30372 30372


In [54]:
def compute_metrics(predictions):
    final_predictions = []
    actual_y_test = []

    current_entry = ''
    counter = 0
    total_counts = 0
    start_index = 0

    if len(predictions) == len(testing_data):
        temp = np.zeros(NUM_CLASSES)
        for i in range(len(predictions)):
            if current_entry != testing_data[start_index+i][0]:
                #compute prev
                if i!=0:
                    temp /= counter
                    final_predictions.append(temp)

                #reset
                total_counts += counter
                counter = 1
                temp = np.zeros(NUM_CLASSES)

                #init new
                current_entry = testing_data[start_index+i][0]
                temp += np.array(predictions[i])
                actual_y_test.append(testing_data[start_index+i][2])
            else:
                temp += np.array(predictions[i])
                counter += 1

        total_counts += counter
        temp /= counter
        final_predictions.append(temp)

    else:
        print('Lengths of predictions dont match with test data')
    
    final_predictions = np.array(final_predictions, dtype=float)
    actual_y_test = np.array(actual_y_test, dtype=float)
    
    #Computing maximal F1-score
    fmax = 0.0
    tmax = 0.0
    precisions = []
    recalls = []
    
    for t in tqdm.tqdm(range(0, 101)):
        
        threshold = t / 100.0
        pred_annots = []
        real_annots = []
    
        for i in range(len(final_predictions)):
            new_preds = []
            new_ys = []
            for j in range(NUM_CLASSES):
                if final_predictions[i][j]>=threshold:
                    new_preds.append(go_terms_bp[j]) #GO_TERMS_BP
                if actual_y_test[i][j]==1:
                    new_ys.append(go_terms_bp[j])
            pred_annots.append(new_preds)
            real_annots.append(new_ys)
    
        fscore, prec, rec, fps, fns = evaluate_annotations(real_annots, pred_annots)
        avg_fp = sum(map(lambda x: len(x), fps)) / len(fps)
        print(f'{avg_fp}')
        precisions.append(prec)
        recalls.append(rec)
        print(f'Fscore: {fscore}, Precision: {prec}, Recall: {rec}, threshold: {threshold}')
        if fmax < fscore:
            fmax = fscore
            tmax = threshold
    print('\nFinal Results (Maximal F1-score):')
    print(f'Fmax: {fmax:0.3f}, threshold: {tmax}')
    precisions = np.array(precisions)
    recalls = np.array(recalls)
        
    
#     rec = recall(actual_y_test,final_predictions)
#     prec = precision(actual_y_test,final_predictions)
#     f1 = f1_score(actual_y_test,final_predictions)
    
#     return rec,prec,f1

In [31]:

recs = []
precs = []
f1s = []

for i in range(1):
    
#     print('Splitting into train-test...')
#     X_train_ng, y_train, X_test_ng, y_test, start_index, prev_index1 = train_test_split(X_ng,y,i,prev_index,Kfolds)
#     X_train_sg, _, X_test_sg, _, _, _= train_test_split(X_sg,y,i,prev_index,Kfolds)
    
#     prev_index = prev_index1
    
    print('Tokenizing...')
    X_train_ng, X_test_ng, vocab_size_ng, tokenizer1 = tokenization(X_train_ng, X_test_ng, MAX_LEN_NG)  
    X_train_sg, X_test_sg, vocab_size_sg, tokenizer2 = tokenization(X_train_sg, X_test_sg, MAX_LEN_SG)
    
   
    print('Shuffling...')   
    shuffled = [[X_train_ng[i],X_train_sg[i],y_train[i]] for i in range(len(X_train_ng))]
    np.random.shuffle(shuffled)

    X_train_ng = [shuffled[i][0] for i in range(len(shuffled))]
    X_train_sg = [shuffled[i][1] for i in range(len(shuffled))]
    y_train = [shuffled[i][2] for i in range(len(shuffled))]
    X_train_ng = np.array(X_train_ng)
    X_train_sg = np.array(X_train_sg)
    y_train = np.array(y_train)
    
    
    model = get_model_ng_sg(vocab_size_ng, vocab_size_sg)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])

    print('Training...')
    history = model.fit([X_train_ng,X_train_sg], y_train, batch_size=32, epochs=10,validation_split=0.2)
    
    print('Evaluating model...')
    predictions = model.predict([X_test_ng,X_test_sg])
    
    print('Computing Metrics...\n')
    compute_metrics(predictions)
    
#     recs.append(rec.numpy())
#     precs.append(prec.numpy())
#     f1s.append(f1.numpy())

# print('Recall:',sum(recs)/len(recs))
# print('Precision:',sum(precs)/len(precs))
# print('F1-Score:',sum(f1s)/len(f1s))

Tokenizing...
Shuffling...


2023-01-27 12:35:06.001710: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30971 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0e:00.0, compute capability: 7.0


Training...
Epoch 1/10


2023-01-27 12:35:18.838508: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


20801/20801 [==============================] - 999s 48ms/step - loss: 0.2328 - recall: 0.7642 - precision: 0.4398 - f1_score: 0.5568 - val_loss: 0.2060 - val_recall: 0.7844 - val_precision: 0.4680 - val_f1_score: 0.5850
Epoch 2/10
20801/20801 [==============================] - 990s 48ms/step - loss: 0.1798 - recall: 0.8191 - precision: 0.4848 - f1_score: 0.6079 - val_loss: 0.1726 - val_recall: 0.8239 - val_precision: 0.4945 - val_f1_score: 0.6171
Epoch 3/10
20801/20801 [==============================] - 991s 48ms/step - loss: 0.1436 - recall: 0.8565 - precision: 0.5270 - f1_score: 0.6515 - val_loss: 0.1520 - val_recall: 0.8502 - val_precision: 0.5194 - val_f1_score: 0.6440
Epoch 4/10
20801/20801 [==============================] - 990s 48ms/step - loss: 0.1190 - recall: 0.8822 - precision: 0.5674 - f1_score: 0.6898 - val_loss: 0.1382 - val_recall: 0.8628 - val_precision: 0.5490 - val_f1_score: 0.6702
Epoch 5/10
20801/20801 [==============================] - 990s 48ms/step - loss: 0.1005

  0%|          | 0/101 [00:00<?, ?it/s]

538.3114624505929
Fscore: 0.04501967559805115, Precision: 0.023028198819250622, Recall: 1.0, threshold: 0.0
96.54229249011858
Fscore: 0.21800932975173246, Precision: 0.12354673248923663, Recall: 0.9260835893181665, threshold: 0.01
71.200790513834
Fscore: 0.2673754006084837, Precision: 0.15666935761301135, Recall: 0.9113688297594668, threshold: 0.02
59.06086956521739
Fscore: 0.29966415141008107, Precision: 0.17970532596510463, Recall: 0.9013288400224042, threshold: 0.03
51.30039525691699
Fscore: 0.3244500190475526, Precision: 0.19827180148303591, Recall: 0.8923025638518547, threshold: 0.04
45.78814229249012
Fscore: 0.34483377706099205, Precision: 0.21406784266566284, Recall: 0.8861480399001559, threshold: 0.05
41.60158102766798
Fscore: 0.3619985708353093, Precision: 0.22785716593331182, Recall: 0.880150441235615, threshold: 0.06
38.09090909090909
Fscore: 0.3785431262438553, Precision: 0.2416258490961261, Recall: 0.8735273779041729, threshold: 0.07
35.31699604743083
Fscore: 0.39283602512

5.705928853754941
Fscore: 0.6305352943323187, Precision: 0.608041114649352, Recall: 0.6547577321254184, threshold: 0.68
5.550197628458498
Fscore: 0.6309819062404549, Precision: 0.6134183002999558, Recall: 0.6495809332544249, threshold: 0.69
5.418181818181818
Fscore: 0.6315808622990917, Precision: 0.6180939464405502, Recall: 0.6456694810495109, threshold: 0.7
5.294861660079051
Fscore: 0.6312223098024939, Precision: 0.6219638726260901, Recall: 0.6407605507932266, threshold: 0.71
5.165217391304348
Fscore: 0.6315482731262545, Precision: 0.6266921368845702, Recall: 0.6364802559228937, threshold: 0.72
5.026877470355731
Fscore: 0.6369248324904475, Precision: 0.6363268975662405, Recall: 0.637523901106604, threshold: 0.73
4.892490118577075
Fscore: 0.6316938879543906, Precision: 0.6363054048410131, Recall: 0.6271487325173559, threshold: 0.74
4.772332015810277
Fscore: 0.6305957819673378, Precision: 0.6407776773912849, Recall: 0.620732404002089, threshold: 0.75
4.632411067193676
Fscore: 0.63087593

In [32]:
model.save('cc_model_0.632.h5')

In [33]:
np.save('cc_predictions',predictions)

In [55]:

recs = []
precs = []
f1s = []

for i in range(1):
    
#     print('Splitting into train-test...')
#     X_train_ng, y_train, X_test_ng, y_test, start_index, prev_index1 = train_test_split(X_ng,y,i,prev_index,Kfolds)
#     X_train_sg, _, X_test_sg, _, _, _= train_test_split(X_sg,y,i,prev_index,Kfolds)
    
#     prev_index = prev_index1
    
    print('Tokenizing...')
    X_train_ng, X_test_ng, vocab_size_ng, tokenizer1 = tokenization(X_train_ng, X_test_ng, MAX_LEN_NG)  
    X_train_sg, X_test_sg, vocab_size_sg, tokenizer2 = tokenization(X_train_sg, X_test_sg, MAX_LEN_SG)
    
   
    print('Shuffling...')   
    shuffled = [[X_train_ng[i],X_train_sg[i],y_train[i]] for i in range(len(X_train_ng))]
    np.random.shuffle(shuffled)

    X_train_ng = [shuffled[i][0] for i in range(len(shuffled))]
    X_train_sg = [shuffled[i][1] for i in range(len(shuffled))]
    y_train = [shuffled[i][2] for i in range(len(shuffled))]
    X_train_ng = np.array(X_train_ng)
    X_train_sg = np.array(X_train_sg)
    y_train = np.array(y_train)
    
    
    model = get_model_ng_sg(vocab_size_ng, vocab_size_sg)
    
    model.compile(
        loss=custom_loss, 
        optimizer='adam', 
        metrics=[
            recall,
            precision,
            f1_score
        ])

    print('Training...')
    history = model.fit([X_train_ng,X_train_sg], y_train, batch_size=32, epochs=10,validation_split=0.2)
    
    print('Evaluating model...')
    predictions = model.predict([X_test_ng,X_test_sg])
    
    print('Computing Metrics...\n')
    compute_metrics(predictions)
    
#     recs.append(rec.numpy())
#     precs.append(prec.numpy())
#     f1s.append(f1.numpy())

# print('Recall:',sum(recs)/len(recs))
# print('Precision:',sum(precs)/len(precs))
# print('F1-Score:',sum(f1s)/len(f1s))

Tokenizing...
Shuffling...
Training...
Epoch 1/10
30256/30256 [==============================] - 1418s 47ms/step - loss: 0.2189 - recall: 0.7759 - precision: 0.4566 - f1_score: 0.5734 - val_loss: 0.1823 - val_recall: 0.8150 - val_precision: 0.4820 - val_f1_score: 0.6047
Epoch 2/10
30256/30256 [==============================] - 1337s 44ms/step - loss: 0.1558 - recall: 0.8417 - precision: 0.5189 - f1_score: 0.6409 - val_loss: 0.1433 - val_recall: 0.8507 - val_precision: 0.5527 - val_f1_score: 0.6691
Epoch 3/10
30256/30256 [==============================] - 1392s 46ms/step - loss: 0.1194 - recall: 0.8820 - precision: 0.5807 - f1_score: 0.6994 - val_loss: 0.1194 - val_recall: 0.8738 - val_precision: 0.6121 - val_f1_score: 0.7191
Epoch 4/10
30256/30256 [==============================] - 1408s 47ms/step - loss: 0.0972 - recall: 0.9067 - precision: 0.6323 - f1_score: 0.7443 - val_loss: 0.1054 - val_recall: 0.8997 - val_precision: 0.6227 - val_f1_score: 0.7352
Epoch 5/10
30256/30256 [=========

  0%|          | 0/101 [00:00<?, ?it/s]

538.3114624505929
Fscore: 0.04501967559805115, Precision: 0.023028198819250622, Recall: 1.0, threshold: 0.0
90.12569169960474
Fscore: 0.2328536250133158, Precision: 0.13324655592727805, Recall: 0.9223370057526786, threshold: 0.01
67.26482213438736
Fscore: 0.280920447505719, Precision: 0.16619320836465634, Recall: 0.9071444912147417, threshold: 0.02
55.85928853754941
Fscore: 0.31403485293347505, Precision: 0.19036987551485424, Recall: 0.8962276769244759, threshold: 0.03
48.64901185770751
Fscore: 0.3391831719237062, Precision: 0.20960889658219228, Recall: 0.888313198764048, threshold: 0.04
43.40632411067194
Fscore: 0.3600853738487087, Precision: 0.2263606815267312, Recall: 0.8798866545489298, threshold: 0.05
39.33122529644269
Fscore: 0.37882094392952254, Precision: 0.24181526012455373, Recall: 0.8740106420965371, threshold: 0.06
36.03478260869565
Fscore: 0.39528862248866875, Precision: 0.255845043575146, Recall: 0.8688261366758872, threshold: 0.07
33.34387351778656
Fscore: 0.409129597229

5.2300395256917
Fscore: 0.6362874038975248, Precision: 0.6317277584823169, Recall: 0.6409133484943372, threshold: 0.68
5.098814229249012
Fscore: 0.6365658622207202, Precision: 0.6369226701394924, Recall: 0.636209453850012, threshold: 0.69
4.972332015810276
Fscore: 0.6356367286217252, Precision: 0.6407689413357742, Recall: 0.6305860751592028, threshold: 0.7
4.831620553359683
Fscore: 0.6354184264894644, Precision: 0.6463976274624391, Recall: 0.6248059644972106, threshold: 0.71
4.714624505928854
Fscore: 0.6359604527835354, Precision: 0.6513455329946238, Recall: 0.6212854065454291, threshold: 0.72
4.583399209486166
Fscore: 0.6360859445019639, Precision: 0.656899036048289, Recall: 0.6165512269497805, threshold: 0.73
4.458498023715415
Fscore: 0.636210857068943, Precision: 0.6627522802344533, Recall: 0.6117134019251221, threshold: 0.74
4.333596837944664
Fscore: 0.6360777822563651, Precision: 0.6684200659521002, Recall: 0.6067208853045984, threshold: 0.75
4.2150197628458494
Fscore: 0.635222376